In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

### A) Download and unzip the file apples.zip.

In [ ]:
image_list = []
raw_masks = []

# load images
image_files = glob.glob("apples/*.jpg")
image_files.sort()
for cur_img in image_files:
    image = plt.imread(cur_img)
    image_list.append(image)
# load masks
mask_files = glob.glob("apples/*.png")
mask_files.sort()
for cur_mask in mask_files:
    raw = plt.imread(cur_mask)
    if raw.shape[-1] == 4:
        raw = np.delete(raw, -1, axis=-1)
    raw_masks.append(raw)

def mask_reshape(data):
    return np.float32(data.reshape(data.shape[0] * data.shape[1], data.shape[2]))

num_images = len(image_list)
total_apple_pixels = np.empty((3, 0))
non_apple_pixels = np.empty((3, 0))
for i in range(num_images):
    reshaped_image = mask_reshape(image_list[i])
    reshaped_mask = np.delete(mask_reshape(raw_masks[i]), [1,2], axis=1).squeeze().astype(int)
    cur_apple_pixels = reshaped_image[reshaped_mask > 0].T / 255.0
    cur_non_apple_pixels = reshaped_image[reshaped_mask == 0].T / 255.0
    total_apple_pixels = np.concatenate((total_apple_pixels, cur_apple_pixels), axis=1)
    non_apple_pixels = np.concatenate((non_apple_pixels, cur_non_apple_pixels), axis=1)

print(total_apple_pixels.shape)
print(non_apple_pixels.shape)

In [ ]:
_, ((ax11, ax12), (ax21, ax22), (ax31, ax32)) = plt.subplots(3, 2)
ax11.imshow(image_list[0])
ax11.set_title('image[0]')
ax12.imshow(raw_masks[0])
ax12.set_title('raw_masks[0]')
ax21.imshow(image_list[1])
ax21.set_title('image[1]')
ax22.imshow(raw_masks[1])
ax22.set_title('raw_masks[1]')
ax31.imshow(image_list[2])
ax31.set_title('image[2]')
ax32.imshow(raw_masks[2])
ax32.set_title('raw_masks[2]')

### B) Use mixtures of Gaussians to distinguish apple vs. non-apple pixels. Use red, green, and blue as your dimensions for now. Make any other decisions you need to, and document them in your .ipynb notebook

In [ ]:
def multivariate_normal_pdf(x, mu, cov):
    if np.linalg.det(cov) == 0:
        cov += 1e-6 * np.identity(cov.shape[0])
    normal_denominator = 1.0 / np.sqrt(np.power((2 * np.pi), x.shape[1]) * np.linalg.det(cov))
    x_mu = np.matrix(x - mu) # (424647, 3)
    x_mu_dot_inv_cov = x_mu @ np.linalg.inv(cov) # (424647, 3)
    exponent = -0.5 * np.sum(np.multiply(x_mu, x_mu_dot_inv_cov), axis=1)
    return (normal_denominator * np.exp(exponent)).reshape(-1)

def getMixGaussLogLike(data, mixGaussEst): 
    """
    Calculate the log likelihood for the whole dataset under a mixture of Gaussians model.

    Keyword arguments:
    data -- d by n matrix containing data points.
    mixGaussEst -- dict containing the mixture of gaussians parameters.

    Returns: 
    logLike -- scalar containing the log likelihood.
    """
    
    data = np.atleast_2d(data)
    # find total number of data items
    nDims, nData = data.shape
    
    # initialize log likelihoods                                                                       
    logLike = 0
                   
    # calculate likelihoods for all data points at once
    likelihoods = np.zeros((mixGaussEst['k'], nData))
    for current_gauss in range(mixGaussEst['k']):
        likelihoods[current_gauss, :] = mixGaussEst['weight'][current_gauss] * multivariate_normal_pdf(data.T, mixGaussEst['mean'][:, current_gauss], mixGaussEst['cov'][:, :, current_gauss])
    
    logLike = np.sum(np.log(np.sum(likelihoods, axis=0)))
    
    return logLike

In [ ]:
def fitMixGauss(data, k):
    nDims, nData = data.shape
    postHidden = np.zeros(shape=(k, nData))

    mixGaussEst = dict()
    mixGaussEst['d'] = nDims
    mixGaussEst['k'] = k
    mixGaussEst['weight'] = (1 / k) * np.ones(shape=(k))
    mixGaussEst['mean'] = 2 * np.random.randn(nDims, k)
    mixGaussEst['cov'] = np.zeros(shape=(nDims, nDims, k))
    for cGauss in range(k):
        mixGaussEst['cov'][:, :, cGauss] = 2.5 + 1.5 * np.random.uniform() * np.eye(nDims)
        
    logLike = getMixGaussLogLike(data, mixGaussEst)
    print('Log Likelihood Iter 0 : {:4.3f}\n'.format(logLike))
    nIter = 30
    for cIter in range(nIter):
        # ===================== =====================
        # Expectation step
        for current_gauss in range(k):
            postHidden[current_gauss, :] = mixGaussEst['weight'][current_gauss] * multivariate_normal_pdf(data.T, mixGaussEst['mean'][:, current_gauss], mixGaussEst['cov'][:, :, current_gauss])
        postHidden = postHidden / np.sum(postHidden, axis=0)

        # ===================== =====================
        # Maximization Step
        for cGauss in range(k):
            mixGaussEst['weight'][cGauss] = np.sum(postHidden[cGauss,:]) / np.sum(postHidden)
            mixGaussEst['mean'][:,cGauss] = np.sum(postHidden[cGauss,:] * data, axis=1) / np.sum(postHidden[cGauss,:])
            
            data_diff = data - mixGaussEst['mean'][:, cGauss][:, np.newaxis]
            mixGaussEst['cov'][:, :, cGauss] = np.dot(data_diff * postHidden[cGauss, :], data_diff.T) / np.sum(postHidden[cGauss, :])

        logLike = getMixGaussLogLike(data, mixGaussEst)
        print('Log Likelihood After Iter {} : {:4.3f}\n'.format(cIter, logLike))


    return mixGaussEst

In [ ]:
# Train
nGaussEst = 6
apple_mog_estimated = fitMixGauss(total_apple_pixels, nGaussEst)
non_apple_mog_estimated = fitMixGauss(non_apple_pixels, nGaussEst)

In [ ]:
# Mean Color of Apple Gaussian Mixture Model
rgb_color_mog_mean = apple_mog_estimated['mean'].T
print(rgb_color_mog_mean)

fig, ax = plt.subplots()
for i in range(len(rgb_color_mog_mean)):
    ax.axhline(i, color=rgb_color_mog_mean[i], linewidth=10)

ax.set_yticks(range(len(rgb_color_mog_mean)))
ax.set_yticklabels([f"Color {i+1}" for i in range(len(rgb_color_mog_mean))])
ax.set_xticks([])
ax.set_title("Mean Color of Apple Gaussian Mixture Model")
plt.show()

In [ ]:
# Mean Color of Non-Apple Gaussian Mixture Model
rgb_color_mog_mean = non_apple_mog_estimated['mean'].T
print(rgb_color_mog_mean)

fig, ax = plt.subplots()
for i in range(len(rgb_color_mog_mean)):
    ax.axhline(i, color=rgb_color_mog_mean[i], linewidth=10)

ax.set_yticks(range(len(rgb_color_mog_mean)))
ax.set_yticklabels([f"Color {i+1}" for i in range(len(rgb_color_mog_mean))])
ax.set_xticks([])
ax.set_title("Mean Color of Non-Apple Gaussian Mixture Model")
plt.show()

### C) Download the file testApples.zip. Generate figures for your notebook, showing each pixel’s posterior probability of being “apple.” Comment on the outcomes

In [ ]:
test_images_list = []

# load test images
test_image_files = glob.glob("testApples/*.jpg")
test_image_files.sort()
for cur_img in test_image_files:
    image = plt.imread(cur_img)
    test_images_list.append(image)

# show test images
_, (ax1, ax2, ax3) = plt.subplots(1, 3)
ax1.imshow(test_images_list[0])
ax1.set_title('test_images_list[0]')
ax2.imshow(test_images_list[1])
ax2.set_title('test_images_list[1]')
ax3.imshow(test_images_list[2])
ax3.set_title('test_images_list[2]')

In [ ]:
prior_apple = 0.4
prior_non_apple = 0.6
threshold = 0.5

posteriro_apple_list = []
for index, cur_img in enumerate(test_images_list):
    print(f"Processing image {index}")
    # TODO
    break

In [ ]:
plt.imshow(posteriro_apple_list[0])

### D) For the test image with a ground-truth mask, quantify and report your result.
Hint:
consider applying a range of thresholds to the posterior to produce sets of {True Positives
(TP), True Negatives (TN), False Positives (FP), and False Negatives (FN)} and using an
ROC curve. Learn about ROC on Wikipedia or see Peter Flach’s chapter on the subject

In [ ]:
# Read the image Bbr98ad4z0A-ctgXo3gdwu8-original.jpg from testApples folder
test_image_with_mask = plt.imread('testApples/Bbr98ad4z0A-ctgXo3gdwu8-original.jpg')
test_gt_mask = plt.imread('testApples/Bbr98ad4z0A-ctgXo3gdwu8-original.png')
_, (ax1, ax2) = plt.subplots(1, 2)
ax1.imshow(test_image_with_mask)
ax1.set_title('test_image_with_mask')
ax2.imshow(test_gt_mask)
ax2.set_title('test_gt_mask')


### E) Download or snap two non-copyrighted photos with apples 
(maybe http://search.creativecommons.org/ or some other source of images that are not copyrighted).
Make good ground-truth masks for them. You can use Windows Paint, or more sophisticated
programs like Gimp (free). Use these as extra test-images. Report your qualitative and
quantitative results.

### F) We should really be using three separate sets of files: a training set, a validation set, and a test set! Explain why